In [1]:
import pandas as pd
import os

In [2]:
from pathlib import Path
from germanetpy.germanet import Germanet
from germanetpy.frames import Frames
from germanetpy.filterconfig import Filterconfig
from germanetpy.synset import WordCategory, WordClass

In [3]:
path2 = "/Users/Richard/PycharmProjects/NeuroLab_3.10/WordList/EmoIma/ESM1.csv"
Emo = pd.read_csv(path2, sep=",")

Emo.head()

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_M_SD,EM_O_F_SD,EM_Y_N,EM_O_N,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,27.38,51.56,18,19,5.83,36.16,36.58,41.77,0.0,0.0
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,34.06,21.04,19,19,-49.37,-62.74,24.86,32.97,0.0,0.0
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,26.29,25.94,18,19,42.83,50.47,27.44,26.39,0.0,0.0
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,22.06,36.59,19,20,28.79,59.00,23.44,29.92,0.0,0.0
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,34.31,25.50,18,19,-39.78,-45.63,23.08,28.66,0.0,0.0


In [4]:
germanet_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/Germanet/GN_V180/GN_V180_XML/"

#frequencylist_nouns = data_path + "/noun_freqs_decow14_16.txt"
germanet = Germanet(germanet_path)

Load GermaNet data...: 100%|█████████▉| 99.99999999999996/100 [00:04<00:00, 20.95it/s] 
Load Wictionary data...: 100%|██████████| 100.0/100 [00:00<00:00, 2009.77it/s]
Load Ili records...: 100%|██████████| 100.0/100 [00:00<00:00, 308631.64it/s]


## Idea: Categorise words based on nature by identifying if the words share a similar hypernym-node 

In [5]:
#[Synset(id=s39164, lexunits=Aal),
 #Synset(id=s49868, lexunits=Aal, Aalfisch, Flussaal)]

### Inspect synsets based on words and synset id 

In [6]:
test_cases = ['Adler', 'Admiral', 'Affe', 'Alge', 'Amateur', 'Angeber']

word = 'Weißkohl'
id = 's73387'

word_synsets = germanet.get_synsets_by_orthform(word)#[0]
synset_id = germanet.get_synset_by_id(id)

#Hypernyms = word_synsets.all_hypernyms()
Hypernyms = synset_id.all_hypernyms()

#synset_id
Hypernyms
#word_synsets

#Synset(id=s34063, lexunits=Mensch, Person, Persönlichkeit, Individuum, Menschenwesen)
#Synset(id=s34063, lexunits=Mensch, Person, Persönlichkeit, Individuum, Menschenwesen)

{Synset(id=s44960, lexunits=Pflanze, Gewächs),
 Synset(id=s44975, lexunits=Samenpflanze),
 Synset(id=s44976, lexunits=Bedecktsamer),
 Synset(id=s45187, lexunits=Zweikeimblättriges Gewächs, Zweikeimblättrige),
 Synset(id=s45188, lexunits=Freiblumenblättriges Gewächs),
 Synset(id=s45352, lexunits=Kreuzblütler, Kreuzblütengewächs),
 Synset(id=s50980, lexunits=Entität),
 Synset(id=s50981, lexunits=Objekt),
 Synset(id=s50982, lexunits=natürliches Objekt),
 Synset(id=s50986, lexunits=Kreatur, Wesen, Wesenheit),
 Synset(id=s50990, lexunits=Lebewesen, Organismus),
 Synset(id=s51001, lexunits=GNROOT),
 Synset(id=s73379, lexunits=Kohl, Kohlpflanze)}

In [47]:
#load Emo
path2 = "/Users/Richard/PycharmProjects/NeuroLab_3.10/WordList/EmoIma/ESM1_categorised.csv"
Emo_cat = pd.read_csv(path2, sep=",")

Emo_cat[Emo_cat["WORD"] == "Sonne"]



,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_N,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group
2040,Sonne,11,8,11,12,93.73,96.13,98.18,91.67,7.98,...,19,77.84,78.79,19.8,22.1,0.0,0.0,NaN,5,short_word


In [59]:
#Berg, Fluß, Sonne exist in Emo df
word = "Fluß"
synsets = germanet.get_synsets_by_orthform(word)
print(synsets)

hypernyms_test = synsets[0].all_hypernyms()
hypernyms_test

#Synset(id=s40348, lexunits=natürlicher Gegenstand)
#Synset(id=s40794, lexunits=natürliches Phänomen)
#Synset(id=s43429, lexunits=Gewässer, Wasser)

[Synset(id=s43463, lexunits=Fluss, Strom), Synset(id=s73673, lexunits=Fluss, Fortgang, Fortlauf)]


{Synset(id=s41396, lexunits=Stelle, Ort, Stätte),
 Synset(id=s41424, lexunits=dimensionsspezifische Stelle),
 Synset(id=s41569, lexunits=Fläche, Gebiet, Feld),
 Synset(id=s42914, lexunits=Gegend, Gefilde, Land),
 Synset(id=s43429, lexunits=Gewässer, Wasser),
 Synset(id=s51001, lexunits=GNROOT)}

In [8]:

# loop for identifying if a single word (synset_id) is nature based human, nature based non-human or neither 
def find_synset_by_id(id):
    try:
        word_synsets = germanet.get_synset_by_id(id)
        hypernyms = word_synsets.all_hypernyms()
    except:
        return False
    
    has_s34063 = False
    has_s50990 = False
    
    for synset in hypernyms:
        if synset.id == "s34063":
            has_s34063 = True
        if synset.id == "s50990":
            has_s50990 = True
        
    if has_s34063 == True and has_s50990 == True:
        return "nature_based_human"
    elif has_s34063 == False and has_s50990 == True:
        return "nature_based_non_human"
    else:
        return ""

find_synset_by_id("s50142")


'nature_based_non_human'

In [9]:
word = "Admiral"
synsets = germanet.get_synsets_by_orthform(word)
print(synsets)

#hypernyms_test = synsets[1].all_hypernyms()
#hypernyms_test

#find_synset_by_id(synsets.id)
find_synset_by_id(synsets[1].id)

[Synset(id=s49365, lexunits=Admiral), Synset(id=s36988, lexunits=Admiral, Admiralin)]


'nature_based_human'

In [10]:
# loop through each synset of a given word to identify if one of the synsets is nature based 
def categorise_synset(word):
    synsets = germanet.get_synsets_by_orthform(word)
    has_human_meaning = False
    has_non_human_meaning = False
    
    for synset in synsets:
        result = find_synset_by_id(synset.id)
        if result == 'nature_based_human':
            has_human_meaning = True
        if result == 'nature_based_non_human':
            has_non_human_meaning = True
            
    if has_human_meaning == True and has_non_human_meaning == True:
        return "nature_based_non_human"
    elif has_human_meaning == False and has_non_human_meaning == True:
        return "nature_based_non_human"
    elif has_human_meaning == True and has_non_human_meaning == False:
        return "nature_based_human"
    else:
        return ""
    
        
print(categorise_synset("Affe"))

nature_based_non_human


In [81]:
# testcases and automated testing
test_cases = ['Ratte', 'Schlange', 'Aal', 'Adler', 'Admiral', 'Affe', 'Alge', 'Amateur', 'Angeber', 'Tisch', 'Kaffee']

for test in test_cases:
    print(f"Result for {test}: {categorise_synset(test)}")

Result for Ratte: nature_based_non_human
Result for Schlange: nature_based_non_human
Result for Aal: nature_based_non_human
Result for Adler: nature_based_non_human
Result for Admiral: nature_based_non_human
Result for Affe: nature_based_non_human
Result for Alge: nature_based_non_human
Result for Amateur: nature_based_human
Result for Angeber: nature_based_human
Result for Tisch: 
Result for Kaffee: 


In [82]:
# create a new column in the Emo df and categorise words 
Emo["Kategorie"] = Emo["WORD"].apply(categorise_synset)
Emo[["WORD", "Kategorie"]].head(50)

,WORD,Kategorie
0,Aal,nature_based_non_human
1,Aas,
2,Abend,
3,Abendbrot,
4,Abfall,
5,Abgrund,
6,Abhang,
7,Abnahme,
8,Abrechnung,
9,Abruf,


In [83]:
counts = Emo["Kategorie"].value_counts()

print(counts)

Kategorie
                          1969
nature_based_non_human     341
nature_based_human         282
Name: count, dtype: int64


In [91]:
#Inspect result in excel
path = '/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/'
file_xlsx = os.path.join(path, "Emo_categorised.xlsx")
file_csv = os.path.join(path, "Emo_categorised.csv")

#Emo.to_excel(file, index=False)
Emo.to_csv(file_csv, index=False)

In [85]:
os.getcwd()

'/Users/Richard/PycharmProjects/NeuroLab_3.10/Germanet/Code'

In [200]:
out_path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/WordList/EmoIma/ESM1_categorised.csv"

Emo.to_csv(out_path, index=False)

In [199]:
Emo.head()

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_O_F_SD,EM_Y_N,EM_O_N,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie
0,Aal,11,10,12,10,90.55,93.40,97.33,90.50,12.30,...,51.56,18,19,5.83,36.16,36.58,41.77,0.0,0.0,nature_based_non_human
1,Aas,10,10,13,11,73.10,57.50,80.69,59.64,20.44,...,21.04,19,19,-49.37,-62.74,24.86,32.97,0.0,0.0,None
2,Abend,10,11,10,12,53.80,52.27,59.90,74.08,28.96,...,25.94,18,19,42.83,50.47,27.44,26.39,0.0,0.0,None
3,Abendbrot,9,11,13,10,64.89,62.64,84.85,82.50,18.72,...,36.59,19,20,28.79,59.00,23.44,29.92,0.0,0.0,None
4,Abfall,8,10,13,9,73.38,77.80,67.46,60.89,28.84,...,25.50,18,19,-39.78,-45.63,23.08,28.66,0.0,0.0,None


In [198]:
Emo.drop(columns=['Path_distance', 'Path_distance_string'], inplace=True)

## Categorization for Plants vs. Animals vs. natural_entity


In [20]:
word = "Affe"

word_synsets = germanet.get_synsets_by_orthform(word)[1]
Hypernyms = word_synsets.all_hypernyms()

word_synsets
Hypernyms

#Synset(id=s48805, lexunits=Tier),

{Synset(id=s48805, lexunits=Tier),
 Synset(id=s48873, lexunits=Gewebetier),
 Synset(id=s49800, lexunits=Chordatier),
 Synset(id=s49812, lexunits=Wirbeltier, Vertebrat),
 Synset(id=s50498, lexunits=Säugetier, Säuger),
 Synset(id=s50519, lexunits=Höheres Säugetier, Plazentatier),
 Synset(id=s50586, lexunits=Herrentier, Primat),
 Synset(id=s50980, lexunits=Entität),
 Synset(id=s50981, lexunits=Objekt),
 Synset(id=s50982, lexunits=natürliches Objekt),
 Synset(id=s50986, lexunits=Kreatur, Wesen, Wesenheit),
 Synset(id=s50990, lexunits=Lebewesen, Organismus),
 Synset(id=s50997, lexunits=höheres Lebewesen),
 Synset(id=s51001, lexunits=GNROOT)}

In [10]:
word = 'Weißkohl'
id = 's73387'

snys = germanet.get_synset_by_id(id)
hypernyms = snys.all_hypernyms()

hypernyms


{Synset(id=s44960, lexunits=Pflanze, Gewächs),
 Synset(id=s44975, lexunits=Samenpflanze),
 Synset(id=s44976, lexunits=Bedecktsamer),
 Synset(id=s45187, lexunits=Zweikeimblättriges Gewächs, Zweikeimblättrige),
 Synset(id=s45188, lexunits=Freiblumenblättriges Gewächs),
 Synset(id=s45352, lexunits=Kreuzblütler, Kreuzblütengewächs),
 Synset(id=s50980, lexunits=Entität),
 Synset(id=s50981, lexunits=Objekt),
 Synset(id=s50982, lexunits=natürliches Objekt),
 Synset(id=s50986, lexunits=Kreatur, Wesen, Wesenheit),
 Synset(id=s50990, lexunits=Lebewesen, Organismus),
 Synset(id=s51001, lexunits=GNROOT),
 Synset(id=s73379, lexunits=Kohl, Kohlpflanze)}

In [21]:
#Berg, Fluß, Sonne exist in Emo df
word = "Fluß"
synsets = germanet.get_synsets_by_orthform(word)
print(synsets)

hypernyms_test = synsets[0].all_hypernyms()
hypernyms_test

#Synset(id=s40348, lexunits=natürlicher Gegenstand)
#Synset(id=s40794, lexunits=natürliches Phänomen)
#Synset(id=s43429, lexunits=Gewässer, Wasser)

[Synset(id=s43463, lexunits=Fluss, Strom),
 Synset(id=s73673, lexunits=Fluss, Fortgang, Fortlauf)]

In [61]:
# loop for identifying if a single word (synset_id) is either a animal, plant or other
#Synset(id=s48805, lexunits=Tier),
test_cases = ["s73387", "s50595", "s42888", "s43463"] #(Weißkohl, Affe, Sonne, Fluss)
def find_synset_by_id_plant_animal(id):
    try:
        word_synsets = germanet.get_synset_by_id(id)
        hypernyms = word_synsets.all_hypernyms()
    except:
        return False
    
    has_s44960 = False #plant
    has_s48805 = False #animal
    has_natural_entity = False 
    
    for synset in hypernyms:
        if synset.id == "s44960":
            has_s44960 = True
        if synset.id == "s48805":
            has_s48805 = True
        if synset.id == "s40348" or synset.id == "s40794" or synset.id == "s43429":
            has_natural_entity = True
        
    if has_s44960 == True and has_s48805 == False:
        return "plant"
    elif has_s44960 == False and has_s48805 == True:
        return "animal"
    elif has_natural_entity == True and has_s44960 == False and has_s48805 == False:
        return "natural_entity"
    else:
        return "error"

#find_synset_by_id_plant_animal("s73387") #Weißkohl

for id in test_cases:
    print(f"Result for {id}: {find_synset_by_id_plant_animal(id)}")

Result for s73387: plant
Result for s50595: animal
Result for s42888: error
Result for s43463: natural_entity


In [44]:
test = germanet.get_synsets_by_orthform("Sonne")[1]
test.all_hypernyms()

{Synset(id=s42867, lexunits=Himmelskörper, Gestirn, Himmelsobjekt),
 Synset(id=s42874, lexunits=Stern),
 Synset(id=s42887, lexunits=Fixstern),
 Synset(id=s50980, lexunits=Entität),
 Synset(id=s50981, lexunits=Objekt),
 Synset(id=s51001, lexunits=GNROOT),
 Synset(id=s69220, lexunits=Zentralgestirn)}

In [29]:
# loop through each synset of a given word to identify if one of the synsets is nature based 
def categorise_synset_plant_animal(word):
    synsets = germanet.get_synsets_by_orthform(word)
    is_plant = False
    is_animal = False
    is_other = False
    
    
    for synset in synsets:
        result = find_synset_by_id_plant_animal(synset.id)
        if result == 'plant':
            is_plant = True
            break
            
        elif result == 'animal':
            is_animal = True
            break
            
        elif result == 'other':
            is_other = True
            
    if is_plant == True:
        return "plant"
    elif is_animal == True:
        return "animal"
    elif is_other == True:
        return "other"
    else:
        return ""
    
        
categorise_synset_plant_animal("Affe")

'animal'

In [31]:
# testcases and automated testing
test_cases = ["Weißkohl", "Affe", "Sonne", "Fluss", "Tisch"]

for test in test_cases:
    print(f"Result for {test}: {categorise_synset_plant_animal(test)}")

Result for Weißkohl: plant
Result for Affe: animal
Result for Sonne: other
Result for Fluss: other
Result for Tisch: other


In [34]:
Emo_cat_nature = Emo_cat[Emo_cat["Kategorie"] == "nature_based_non_human"]

In [40]:
Emo_cat_nature["Kategorie2"] = Emo_cat_nature["WORD"].apply(categorise_synset_plant_animal)
Emo_cat_nature[["WORD", "Kategorie2"]]
Emo_cat_nature[Emo_cat_nature["Kategorie2"] == "other"]

Emo_cat_nature[["WORD", "Kategorie2"]]

/var/folders/jz/fzhqjtjs2qs1vdhm2p0k6mpm0000gn/T/ipykernel_5014/3359831763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Emo_cat_nature["Kategorie2"] = Emo_cat_nature["WORD"].apply(categorise_synset_plant_animal)


,WORD,Kategorie2
0,Aal,animal
20,Adler,animal
21,Admiral,animal
22,Affe,animal
26,Ahorn,plant
...,...,...
2533,Zeder,plant
2551,Ziege,animal
2553,Ziegenbock,animal
2565,Zobel,animal


In [41]:
len(Emo_cat[(Emo_cat["Is_plant"] == "plant") & (Emo_cat["length_group"] == "short_word")])

70

In [42]:
#Inspect result in excel
path = '/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/lists'
#file_xlsx = os.path.join(path, "Emo_categorised.xlsx")
file_csv = os.path.join(path, "Emo_categorised.csv")

#Emo.to_excel(file, index=False)
Emo_cat.to_csv(file_csv, index=False)

In [43]:
#filter for short plants 
Emo_cat_short_plants = Emo_cat[(Emo_cat["Is_plant"] == "plant") & (Emo_cat["length_group"] == "short_word")]
Emo_cat_short_plants

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,Is_plant
26,Ahorn,12,8,13,10,85.67,70.38,88.85,78.50,12.18,...,33.50,44.53,31.95,26.68,0.00,0.00,nature_based_non_human,5,short_word,plant
34,Alge,8,11,13,10,67.00,75.55,76.23,84.70,19.86,...,-5.94,2.11,24.93,36.10,0.00,0.00,nature_based_non_human,4,short_word,plant
51,Ananas,12,8,11,12,93.00,95.38,97.82,93.08,10.89,...,39.33,55.47,28.43,26.28,0.00,0.00,nature_based_non_human,6,short_word,plant
83,Apfel,10,10,12,11,94.00,97.60,97.17,92.45,6.39,...,23.50,68.00,27.64,26.50,0.00,0.00,nature_based_non_human,5,short_word,plant
155,Bambus,11,10,11,12,86.09,86.50,84.36,84.50,13.02,...,35.11,30.95,33.37,31.76,0.00,0.00,nature_based_non_human,6,short_word,plant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,Wein,9,11,10,13,81.00,91.64,92.10,89.23,18.02,...,30.72,42.16,46.79,33.77,0.00,0.00,nature_based_non_human,4,short_word,plant
2455,Weizen,13,8,12,10,76.69,81.00,93.67,90.90,15.16,...,26.72,38.37,21.61,27.50,0.00,0.00,nature_based_non_human,6,short_word,plant
2481,Winde,9,9,12,11,70.44,60.11,64.67,63.55,26.92,...,16.78,-4.42,31.16,38.14,0.04,0.00,nature_based_non_human,5,short_word,plant
2515,Wurzel,9,10,11,12,83.89,89.60,85.18,82.00,11.11,...,13.32,5.89,27.73,26.08,0.00,0.00,nature_based_non_human,6,short_word,plant


In [44]:
#filter for long plant words 
Emo_cat_long_plants = Emo_cat[(Emo_cat["Is_plant"] == "plant") & (Emo_cat["length_group"] == "long_word")]
Emo_cat_long_plants

,WORD,IM_Y_M_N,IM_Y_F_N,IM_O_M_N,IM_O_F_N,IM_Y_M_MEAN,IM_Y_F_MEAN,IM_O_M_MEAN,IM_O_F_MEAN,IM_Y_M_SD,...,EM_Y_MEAN,EM_O_MEAN,EM_Y_SD,EM_O_SD,F_Unknown_Y,F_Unknown_O,Kategorie,WORD_length,length_group,Is_plant
84,Apfelbaum,11,9,11,12,88.09,88.67,98.27,92.58,12.24,...,48.61,50.68,29.13,24.72,0.00,0.00,nature_based_non_human,9,long_word,plant
179,Baumwolle,8,12,13,10,69.13,79.17,84.54,82.20,23.76,...,34.72,29.42,23.03,23.54,0.00,0.00,nature_based_non_human,9,long_word,plant
261,Blumenkohl,11,10,12,11,86.00,91.20,96.83,94.00,13.41,...,16.05,45.79,24.67,28.50,0.00,0.00,nature_based_non_human,10,long_word,plant
299,Brombeere,10,10,12,11,88.60,94.60,93.83,85.91,16.46,...,36.44,54.68,25.30,27.73,0.00,0.00,nature_based_non_human,9,long_word,plant
428,Edelweiß,11,8,11,12,66.45,68.88,98.00,88.58,36.90,...,26.94,54.58,32.71,26.74,0.00,0.00,nature_based_non_human,8,long_word,plant
599,Flieder,10,11,12,11,75.70,86.09,94.67,91.00,22.11,...,34.17,58.32,18.59,22.62,0.00,0.00,nature_based_non_human,7,long_word,plant
734,Getreide,10,9,13,10,74.80,75.89,91.23,90.00,16.36,...,26.16,32.42,28.41,21.10,0.00,0.00,nature_based_non_human,8,long_word,plant
747,Ginster,11,7,11,11,39.00,72.71,76.82,80.91,30.93,...,16.12,45.32,19.04,32.15,0.07,0.04,nature_based_non_human,7,long_word,plant
850,Haselnuss,11,10,14,9,89.36,86.50,95.50,94.78,10.56,...,20.94,42.26,27.79,31.33,0.00,0.00,nature_based_non_human,9,long_word,plant
991,Kamille,10,10,12,11,70.00,86.20,80.67,85.36,20.48,...,32.05,42.42,25.53,21.75,0.00,0.00,nature_based_non_human,7,long_word,plant
